[![](./media/Db2_header_3.png)](https://www.ibm.com/demos/collection/db2-database/)
<a id="top">

In [1]:
%run refresh.ipynb

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Containerizing Db2

This lab takes you through the steps move a Db2 Database from an on premises installation to a containerized Db2 Database running on RedHat OpenShift.

https://www.ibm.com/cloud/blog/how-to-build-a-db2-for-openshift-environment-for-your-desktop

Take a few minutes to watch this overview video. It includes a high level story of the benefits of containerization as well as the basics of how easy it is to containerize Db2.

In [23]:
%%html 
<iframe width="560" height="315" src="https://www.youtube.com/embed/czfmfZNb8Gg" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

## Using the Lab Environment

### Keeping the terminal window visible
Part of the lab use the Linux terminal to issue commands. It may be easier to keep the terminal window on top of the Jupyter notebook when running these commands. When you have a terminal window displayed, **right click on the title bar** and select **Always on Top** to keep the screen visible during the duration of the lab.

### Organizing your Browser Windows
In parts of the lab, you work with the Db2 Console and the Red Hat OpenShift Console while following instructions in this page. When instructed, open the new browser pages in a new page and organize your browser windows so that you can see both the console as well as the lab instructions. 

### How to Copy Code and Examples
Throughout this lab there are code samples that need to be copied and modified in a text editor. Any commands that need to be executed from a command line are found in grey boxes (an example is found below) has been designed to be easily copied.

In [12]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black;" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre>
Sample commands are found in cells like this.
</pre>
</div>

Highlight the text and copy it using **Copy** from the right click menu or type Ctrl-C. Paste the command into the Command terminal using **Paste** from the right click menu or type Ctrl-V.

## Background to our OpenShift Deployment Environment

The Deployment environment is a CentOS 7.7 Operating system with Openshift 3.11 Single node Install and Helm 2.16 deployed with an installation of Tiller.

If you are interested in deploying such an environment yourself for your own use you can find the instructions at: 
    

In [13]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black;" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre>
https://www.ibm.com/cloud/blog/how-to-build-a-db2-for-openshift-environment-for-your-desktop
</pre>
</div>

## Exploring the Db2 Moderization Playground Environment

The next steps walk you through the preconfigured Db2 Modernization Environment. 

There are four virtual machines or nodes in this environment. All are running on Centos 7 virtual machines.
* host-1 (10.0.0.1) **Red Hat OpenShift with Db2 11.5.4.0 Cartridge**
    * Main UserID: db2pot
    * password: 123qwe123
* server7 (10.0.0.2) **Db2 11.1 on Premises installation**
    * Main UserID: db2inst1
    * password: db2inst1
* host-2 (10.0.0.3) **Db2 Data Management Console 3.1.3 and Db2 11.5 Repository Database**
    * Main UserID: db2inst1
    * password: db2inst1
* host-3 (10.0.0.4) **Jupyter Notebook Environment**
    * Main UserID: ibmuser
    * password: engageibm
  
<img src="./media/DemonstrationEnvironment.png">

### Working with the Virtual Machines in the Demonstration Cluster
To switch between machines click on the machine you want to work with and log in using the provided user id. 

## Explore a Db2 Cartridge running on Red Hat OpenShift
The Db2 Cartridge is Db2 running in a container. The following steps walk through how to explore the pre-configured Red Hat OpenShift environment that includes two Db2 Cartridges actively running. You can work with OpenShift either through the command line or the OpenShift Web Console. Start by logging into OpenShift running on machine host-1 using the OpenShift command line interface (OC).

### Log into OpenShift from the Command Line
1. You should already be using the **Db2 Source Data Server** Virtual Machine 
<img src="./media/Db2SourceDataServer.png">
2. If you have not already done so, log in with the **db2inst1** userid and **db2inst1** password
3. Double-Click the **Terminal** icon
<img src="./media/TerminalIcon.png">
4. From the db2inst1@server7 prompt enter

In [15]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black;" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre>
oc login host-1:8443 -u admin -n db2
</pre>
</div>

Now, enter the following password to log in to OpenShift using the OC command line processor running on Server7.

In [16]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black;" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre>
ocadmin
</pre>
</div>

OC needs to be installed on the same machine as the source database you plan to containerize. It is a pre-requiset for the Db2 Containerization tool. The -u parameter defines the user to log in with, the -n parameter defines the project to use. 

### Review the Status of the Db2 Cartridges running on OpenShift
Run the following command to see the status of all the Db2 OpenShift Pods. You should see the Pods used to start Db2 (Completed) and those currently running.  

In [17]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black;" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre>
oc get pods
</pre>
</div>

    [db2inst1@server7 ~]$ oc get pods
    NAME                                 READY     STATUS      RESTARTS   AGE
    db2u-db2u-0                          1/1       Running     13         16d
    db2u-db2u-engn-update-job-82h8g      0/1       Completed   0          16d
    db2u-db2u-ldap-59b74ddc8d-x6hcp      1/1       Running     13         16d
    db2u-db2u-nodes-cfg-job-x7nzm        0/1       Completed   0          16d
    db2u-db2u-restore-morph-job-hj8hv    0/1       Completed   0          16d  
    db2u-db2u-sqllib-shared-job-dbxf4    0/1       Completed   0          16d
    db2u-db2u-tools-599ff967b-6fqxn      1/1       Running     13         16d
    db2u-etcd-0                          1/1       Running     13         16d
    db2u-etcd-1                          1/1       Running     13         16d
    db2u-etcd-2                          1/1       Running     13         16d
    db2u2-db2u-0                         1/1       Running     8          9d
    db2u2-db2u-engn-update-job-dbfdk     0/1       Completed   0          9d
    db2u2-db2u-ldap-56b985d998-lm6qh     1/1       Running     8          9d
    db2u2-db2u-nodes-cfg-job-t4nvm       0/1       Completed   0          9d
    db2u2-db2u-restore-morph-job-wfq2x   0/1       Completed   0          9d
    db2u2-db2u-sqllib-shared-job-x2m25   0/1       Completed   0          9d
    db2u2-db2u-tools-7f85b6fb94-rdw6f    1/1       Running     8          9d
    db2u2-etcd-0                         1/1       Running     8          9d
    db2u2-etcd-1                         1/1       Running     8          9d
    db2u2-etcd-2                         1/1       Running     8          9d

Now issue the following command to see the **db2u** (Db2 Carridge) service. The get svc command shows you how OpenShift translates the Db2 50000 and 50001 ports to 32019 and 32149 to make them accessible outside of OpenShift. We need to be able to access one of these ports to move an on premises copy of Db2 into Db2 on OpenShift.

In [18]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black;" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre>
oc get svc db2u-db2u-engn-svc
</pre>
</div>

    [db2inst1@server7 ~]$ oc get svc db2u-db2u-engn-svc
    NAME                 TYPE       CLUSTER-IP     EXTERNAL-IP   PORT(S)                           AGE
    db2u-db2u-engn-svc   NodePort   172.30.70.21   <none>        50000:32019/TCP,50001:32149/TCP   16d


### Log into the OpenShift Web Console
You can see the same detail through the OpenShift Console that you just saw from the OC command line interface.

1. Right click on the following link and select **Open Link in New Window**: https://host-1:8443/console/project/db2/overview
3. Accept the security warning
4. Log into the console using **admin** and password **ocadmin**
<img src="./media/OpenShiftConsole.png">  
5. Click **Overview** and scan down the active applications. Two have been setup: db2u and db2u2
6. Click **Applications** and then **Services**
<img src="./media/OpenShiftConsoleServices.png">
7. Scroll down to **db2u-db2u-engn-svc**
8. Click **db2u-db2u-engn-svc**
9. Scroll down to the **Traffic** table. You can see that the Db2 50000 port is mapped to 30781 on the **host-1** OpenShift machine.
<img src="./media/TrafficPorts.png"> 

### Log into the Db2 Data Management Console

1. Right click on the following link and select **Open Link in New Window**: http://10.0.0.3:11080
4. Log into the console using **admin** and password **passw0rd**
<img src="./media/DMCDatabases.png"> 
5. Hover your mouse pointer over **Db2_On_Native_Linux**. Notice that this database is at version 11.1.4.5.
<img src="./media/FlyoverDb2Native.png"> 
5. Hover your mouse pointer over **Db2_On_OpenShift**. Notice that this database is at version 11.5.4.0
<img src="./media/FlyoverDb2OpenShift.png"> 
5. Click **Db2_On_Native_Linux**
6. Click the **Explore** icon in the main menu
7. Click **Tables**
<img src="./media/ExploreTables.png"> 
8. Click the checkbox next to **DB2INST1** to see all the user tables on the source database
<img src="./media/DatabaseTables.png"> 
9. Click the down arrow beside **Db2_On_Native_Linux** in the upper right side of the Db2 Console
<img src="./media/DatabaseSelection.png"> 
9. Select **Db2_On_OpenShift** from the database dropdown list. This lets you quickly switch between databases in the Db2 Console.
<img src="./media/DatabaseSelector.png"> 
Notice that the SAMPLE tables under DB2INST1 are not in the empty Db2 Cartridge container. The Db2 on OpenShift does not contain any user tables. 

## Containerize an existing Db2 installation
### Create the Click to Containerize Script to move the Db2_On_Native_Linux database to Openshift

1. Make sure you using server7
2. If you have not already done so, log in with the **db2inst1** userid and **db2inst1** password
3. Double-Click the **c2cdb2** icon
<img src="./media/C2CIcon.png">
4. Click **Agree** to accept the **Disclaimer and User Acceptance** terms
5. Click **Continue**. You do not need to enter an opportunity code, and leave **Prototpe Mode** unchecked.
6. Click the arrow icon in the **Containerize Db2** box
7. For the **Source Database**: Select **On_Premises_Db2** in the profile name dropdown.
8. Enter **db2inst1** as the password for the source database
9. For the **Target OpenShift Container**: Select **OpenShift_Db2_01** in the profile name dropdown.  
10. Enter **redhat** as the password for the target OpenShift Container
11. Enter **admin** as the OpenShift OC Userid
12. Enter **ocadmin** as the OC Password
13. Select **Check Connections**
14. Select **4** as the **Number of Threads**
<img src="./media/CreateScript.png">
15. Click **Analyze** to check out how the tool automatically analyzes the differences between the source database and the target Cartridge.
<img src="./media/AnalysisSummary.png">
16. Click on the Database Parameters Summary box to see details of the new Database Parameters that will be used. 
<img src="./media/DatabaseParameteres.png">
You can override parameters if required.  
17. Click **Close** to return to the **Generate Database Containerization Script** page when you are done exploring.
18. Click **Create Script**
<img src="./media/ScriptCreated.png">
19. Click **OK** to view the deployment information
<img src="./media/RunningShift.png">
20. Click **OK**
21. Click **Exit**

### Run the Click to Containerize Script

1. Make sure you using server7
2. If you have not already done so, log in with the **db2inst1** userid and **db2inst1** password
3. Double-Click the **Terminal** icon
4. From the db2inst1@server7 prompt enter

In [19]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black;" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre>
db2stop force
</pre>
</div>

This will stop the current Db2 server. You should see the following before proceeding:

    [db2inst1@server7 ~]$ db2stop force
    11/06/2020 10:48:15     0   0   SQL1064N  DB2STOP processing was successful.
    SQL1064N  DB2STOP processing was successful.

Now you can run the script generated by Db2 Click to Containerize. You need to switch to the directory containing the script and make sure you have permission to run it.

In [20]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black;" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre>
cd c2cdb2/scripts
</pre>
</div>

In [21]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black;" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre>
chmod 777 shift.sh
</pre>
</div>

In [22]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black;" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre>
./shift.sh
</pre>
</div>

Once the script is finished you can restart the source Db2 database server

In [24]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black;" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre>
db2start
</pre>
</div>

## Check the Db2 Cartrige On Red Hat OpenShift
Now that you have moved the Db2 database from a Version 11.1 native install on Linux to Db2 running in a Version 11.5.0.4 Cartridge on RedHat OpenShift, you can check that all the tables in the source database appear in the replaced container.

1. Open the Db2 Data Management Console and return to the **Tables** screen
<img src="./media/EmptyTables.png">  
2. Make sure that **Db2_On_OpenShift** is still selected in the database dropdown in the upper right of the console screen.
<img src="./media/SelectDB.png">
3. Click the refresh icon. This rechecks the availble tables in the Db2 Cartridge running on OpenShift
<img src="./media/Refresh.png">
4. Notice that the tables originally in the native Linux Db2 database are now in the Db2 Cartridge running on OpenShift. Remember, this isn't just moving tables, the whole database, including the settings have been moved and upgraded.
5. Click **Databases** at the upper left of the console. From this page you can confirm that the original Db2 database on Linux is up and running, as well as the upgraded copy on OpenShift. 
6. Hover your mouse over the database names to confirm the Version and Release of each database. Notice that the Db2 Database is automatically upgraded to 11.5 during the process.

## Next Steps
You can try experimenting with other aspects of the Db2 Containerization tool as well as the Version 3.1.3 Db2 Data Management Console. You can try creating a new database on Server 7 called SAMPLE2 and upgrading it using Db2 Click to Containerize to the second Db2 Cartridge on RedHat OpenShift. You can also try out the Hands-On Labs available on the RedHat OpenShift machine.

### Accessing the Db2 Console and the OpenShift console from the Linux desktop
Icons are available on the Linux desktop with links to:
* The **Db2 Data Management Console**
<img src="./media/Db2DMCIcon.png"> 
* The **OpenShift Console**
<img src="./media/OpenShiftConsoleIcon.png">

#### Accessing Jupyter notebooks and the Db2 Console directly from your own browser
Your lab welcome email includes direct links to the Db2 Data Management Console as well as to the Jupyter notebook environment. You can follow these links from your own desktop browser. However remember that any html links to resources referenced inside a lab (for example http:/10.0.0.2) will only work if you are running on a browser inside the virtual desktop environment.

### We hope you have enjoyed this lab, to view more labs in this collection click on the footer bar below!

[![](./media/db2_dte_footer.png)](https://www.ibm.com/demos/collection/db2-database/)
<a id="top">

#### Credits: IBM 2021, Peter Kohlmann [kohlmann@ca.ibm.com]